In [1]:
import numpy as np
import random
import os

In [2]:
os.chdir('/Users/shubhampatel/Desktop/PRML/Assignment 3/Data')
os.getcwd()

'/Users/shubhampatel/Desktop/PRML/Assignment 3/Data'

In [59]:
def Kmeans(x, μ, K):
	n_tot, d = x.shape
	err = []
	N = np.zeros(K)
	distribution = np.zeros([n_tot, K])
	for n in range(n_tot):
		for k in range(K):
				err.append(np.linalg.norm(μ[k]-x[n]))
		err = np.array(err)
		i = np.argmin(err)
		N[i] += 1
		distribution[n, i] = 1
		err = []

	μ_new = np.zeros([K, d])
	for k in range(K):
		for n in range(n_tot):
			μ_new[k, :] += x[n]*distribution[n, k]
		μ_new[k, :] /= N[k]

	Σ_new = np.zeros([K, d, d])
	distortion = 0
	for k in range(K):
		for n in range(n_tot):
			Σ_new[k, :, :] += distribution[n, k]*np.eye(d)*((x[n]-μ_new[k]).reshape(d,1)@(x[n]-μ_new[k]).reshape(1,d))
			distortion += distribution[n, k]*np.linalg.norm(μ_new[k]-x[n])
		Σ_new[k, :, :] /= N[k]

	return (N, μ_new, Σ_new, distortion)

In [64]:
def gaussian(x, μ, Σ):
	d = len(μ)
	exponent = np.exp(-0.5*(x-μ)@np.linalg.inv(Σ)@(x-μ).T)
	det = 1/(np.linalg.det(Σ))**0.5
	gdf = 1/(2*np.pi)**(d/2)*det*exponent
	return gdf

def GMM(x, π, μ, Σ, K):
	n_tot, d = x.shape
	r = np.zeros((n_tot, K))
	for n in range(n_tot):
		Sum = 0
		for k in range(K):
			Sum += π[k]*gaussian(x[n], μ[k], Σ[k])
		for k in range(K):
			r[n, k] = (π[k]*gaussian(x[n], μ[k], Σ[k]))/Sum

	N = np.zeros(K)
	π_new = np.zeros(K)
	for k in range(K):
		for n in range(n_tot):
			N[k] += r[n, k]
		π_new[k] = N[k]/n_tot
	# print(N)
	μ_new = np.zeros((K, d))
	for k in range(K):
		for n in range(n_tot):
			μ_new[k, :] += r[n, k]*x[n]
		μ_new[k, :] /= N[k]

	Σ_new = np.zeros([K, d, d])
	distortion = 0
	for k in range(K):
		for n in range(n_tot):
			Σ_new[k, :, :] += r[n, k]*np.eye(d)*((x[n]-μ_new[k]).reshape(d,1)@(x[n]-μ_new[k]).reshape(1,d))
			distortion += r[n, k]*np.linalg.norm(μ_new[k]-x[n])
		Σ_new[k, :, :] /= N[k]
#     for K in np.arange(K):
#         Σ_new[K, :, :] = np.eye(d)*Σ_new[K, :, :]

	return (π_new, μ_new, Σ_new, distortion)

In [5]:
def read_data(dataset, type):
	curr = os.getcwd()
	path = f"{curr}/Image Dataset/{dataset}/{type}"
	os.chdir(path)

	dataList = []
	files = [file for file in os.listdir()]

	for file in files:
		dataList.append(np.genfromtxt(file, delimiter=' ', dtype=float))
	os.chdir(curr)
	return np.array(dataList)

In [6]:
train_coast = read_data('coast', 'train')
dev_coast = read_data('coast', 'dev')
train_forest = read_data('forest', 'train')
dev_forest = read_data('forest', 'dev')
train_highway = read_data('highway', 'train')
dev_highway = read_data('highway', 'dev')
train_mountain = read_data('mountain', 'train')
dev_mountain = read_data('mountain', 'dev')
train_opencountry = read_data('opencountry', 'train')
dev_opencountry = read_data('opencountry', 'dev')

In [7]:
C = []

C1 = train_coast.flatten().reshape(-1, 23)
C.append((C1 - C1.min(axis = 0))/(C1.max(axis = 0)-C1.min(axis = 0)))
C2 = train_forest.flatten().reshape(-1, 23)
C.append((C2 - C2.min(axis = 0))/(C2.max(axis = 0)-C2.min(axis = 0)))
C3 = train_highway.flatten().reshape(-1, 23)
C.append((C3 - C3.min(axis = 0))/(C3.max(axis = 0)-C3.min(axis = 0)))
C4 = train_mountain.flatten().reshape(-1, 23)
C.append((C4 - C4.min(axis = 0))/(C4.max(axis = 0)-C4.min(axis = 0)))
C5 = train_opencountry.flatten().reshape(-1, 23)
C.append((C5 - C5.min(axis = 0))/(C5.max(axis = 0)-C5.min(axis = 0)))

In [42]:
C[0].mean()

0.45568908558303745

In [8]:
def stats(x):
	μ = x.mean(0)
	Σ = np.cov(x, rowvar=False)
	return (μ, Σ)

def initial(x, K):
	n, d = x.shape
	μ_initial = np.zeros([K, d])
	Σ_initial = np.zeros([K, d, d])
	for k in range(K):
		index = random.randint(0, n)
		μ_initial[k,:] = x[index]
		Σ_initial[k,:,:] = stats(x)[1]
	return (μ_initial, Σ_initial)

In [162]:
def ROC(xtest, y_true, P):
    
    TPR = []; FPR = []
     
    threshold_min, threshold_max = np.log10(P).min(), np.log10(P).max()
    threshold_vec = np.linspace(threshold_min, threshold_max, dtype=float, num=100)
    for t in threshold_vec:
        TP, FP = 0, 0
        TN, FN = 0, 0
        threshold = 10**(t)
        y_predict = []
        for i in range(len(xtest)):
            if P[i,0]/(P[i,:])>=threshold:
                y_predict.append(1)
            else:
                y_predict.append(2)
        y_predict = np.array(y_predict)

        for i in range(len(xtest)):
            if y_predict[i] == 1 and y_true[i] == 1:
                TP += 1
            elif y_predict[i] == 1 and y_true[i] == 2:
                FP += 1
            elif y_predict[i] == 2 and y_true[i] == 1:
                FN += 1
            else:
                TN += 1

        TPR.append(TP/(TP+FN))
        FPR.append(FP/(FP+TN))
    return (TPR, FPR)

In [144]:
## K Keans implementation
K = 10
μ = []
Σ = []
N = []
dk = []

for c in range(5):
	μ_initial, Σ_initial = initial(C[c], K)
	μ.append(μ_initial)
	Σ.append(Σ_initial)

μ = np.array(μ)
Σ = np.array(Σ)

for c in range(5):
	N_c, μ[c], Σ[c], dk_c = Kmeans(C[c], μ[c], K)
	N.append(N_c)
	dk.append(dk_c)

N = np.array(N)
dk = np.array(dk)

for c in range(5):
	for j in np.arange(1, 15):
		N[c], μ[c], Σ[c], dk[c] = Kmeans(C[c], μ[c], K)
  

In [145]:
π = []
dg = []
for c in range(5):
	π.append(np.zeros(K))

π = np.array(π)

for c in range(5):
	for k in range(K):	
		π[c][k] = N[c][k]/len(C[c])

In [146]:
##GMM implementation
π = []
dg = []
for c in range(5):
	π.append(np.zeros(K))

π = np.array(π)

for c in range(5):
	for k in range(K):	
		π[c][k] = N[c][k]/len(C[c])

for c in range(5):
	π[c], μ[c], Σ[c], dg_c = GMM(C[c], π[c], μ[c], Σ[c], K)
	dg.append(dg_c)

dg = np.array(dg)

for c in range(5):
	for j in range(2):
		π[c], μ[c], Σ[c], dg[c] = GMM(C[c], π[c], μ[c], Σ[c], K)

In [50]:
C_d = []

C1_d = dev_coast.flatten().reshape(-1, 23)
C_d.append((C1_d - C1_d.min(axis = 0))/(C1_d.max(axis = 0)-C1_d.min(axis = 0)))
C2_d = dev_forest.flatten().reshape(-1, 23)
C_d.append((C2_d - C2_d.min(axis = 0))/(C2_d.max(axis = 0)-C2_d.min(axis = 0)))
C3_d = dev_highway.flatten().reshape(-1, 23)
C_d.append((C3_d - C3_d.min(axis = 0))/(C3_d.max(axis = 0)-C3_d.min(axis = 0)))
C4_d = dev_mountain.flatten().reshape(-1, 23)
C_d.append((C4_d - C4_d.min(axis = 0))/(C4_d.max(axis = 0)-C4_d.min(axis = 0)))
C5_d = dev_opencountry.flatten().reshape(-1, 23)
C_d.append((C5_d - C5_d.min(axis = 0))/(C5_d.max(axis = 0)-C5_d.min(axis = 0)))

In [46]:
C_d[0].shape

(2628, 23)

In [161]:
Σ.shape

(5, 10, 23, 23)

In [57]:
from scipy.stats import multivariate_normal

In [ ]:
D = []

D1 = dev_coast.flatten().reshape(-1, 23)
D.append((D1 - D1.min(axis = 0))/(D1.max(axis = 0)-D1.min(axis = 0)))
D2 = dev_forest.flatten().reshape(-1, 23)
D.append((D2 - D2.min(axis = 0))/(D2.max(axis = 0)-D2.min(axis = 0)))
D3 = dev_highway.flatten().reshape(-1, 23)
D.append((D3 - D3.min(axis = 0))/(D3.max(axis = 0)-D3.min(axis = 0)))
D4 = dev_mountain.flatten().reshape(-1, 23)
D.append((D4 - D4.min(axis = 0))/(D4.max(axis = 0)-D4.min(axis = 0)))
D5 = dev_opencountry.flatten().reshape(-1, 23)
D.append((D5 - D5.min(axis = 0))/(D5.max(axis = 0)-D5.min(axis = 0)))

In [164]:
prediction.shape

(2628, 5)

In [163]:
## Confusion Matrix

prediction = np.zeros((C_d[0].shape[0], 5))
score = np.zeros((C_d[0].shape[0], 5))

for j in range(5):
  for i in range(C_d[0].shape[0]):
    pt = C_d[0][i]
    P = np.zeros(5)
    #print(C1_d[i].min())

    for c in range(5):
      for k in range(K):
        P[c] += π[c,k]*gaussian(pt, μ[c,k], Σ[c,k])
        #print((pt-μ[c,k])@np.linalg.inv(Σ[c,k])@(pt-μ[c,k]).T)
        #print(multivariate_normal.pdf(pt, μ[c,k], Σ[c,k]))
        #print((pt-μ[c,k]))
        #print(np.linalg.inv(Σ[c,k]))
        #print(gaussian(pt, μ[c,k], Σ[c,k]))
    #print(P)
    prediction[i,j] = np.argmax(P)
    #score[i,j] = np.log(P.max())

# print(prediction)
# print(score)
# final = np.zeros((C_d[0].shape[0], 5))
# for c in range(5):
#   count = 0
#   for i in range(C_d[0].shape[0]):
#     class_count = np.zeros(5)
#     for j in range(36):
#       class_count[int(prediction[count, c])] += 1
#       count += 1
#     final[i, c] = np.argmax(class_count)

# CMatrix = np.zeros((5,5))
# for c in range(5):
# 	for i in range(C_d[0].shape[0]):
# 		CMatrix[int(final[i, c]), c] +=  1

print(CMatrix)

IndexError: index 2628 is out of bounds for axis 0 with size 2628

In [158]:
fig = plt.figure(figsize=(10, 7))
(TPR, FPR) = ROC(C_d[0], y_true, P)

2628